# 정당성 검정

1. 각 소속사 주가 + 사건 데이터를 읽고 소속사 컬럼 붙여서 하나로 병합
2. 벤치마크 지수도 KOSDAQ_ENT, KOSDAQ 각각 준비
3. 소속사별 + 사건별로:
    - 사건일 ±7일 윈도우 추출
    - 로그수익률 & 초과수익률(AR) 계산
    - CAR 계산
4. 결과 데이터프레임 구성:
    - 소속사, 그룹, 연예인, 사건일, CAR_KOSDAQ_ENT, CAR_KOSDAQ, 사건명
5. 전체 CAR 리스트로 평균 검정 (t-test)

In [5]:
import pandas as pd
import numpy as np
from scipy import stats

# 주가 & 사건 파일 목록
companies = ['HYBE', 'JYP', 'SM', 'YG']
stock_files = {c: f'../데이터/{c}_주가_전처리.csv' for c in companies}
incident_files = {c: f'../데이터/{c}_사건_전처리.csv' for c in companies}

# 벤치마크 파일 (KOSDAQ & 엔터테인먼트)
kosdaq_ent_file = '../데이터/KOSDAQ Entertainment & Culture_전처리.csv'
kosdaq_file = '../데이터/코스닥_전처리.csv'

# 벤치마크 데이터 불러오기
kosdaq_ent = pd.read_csv(kosdaq_ent_file, parse_dates=['날짜'])
kosdaq = pd.read_csv(kosdaq_file, parse_dates=['날짜'])

# 로그수익률 계산 함수
def add_return(df):
    df = df.sort_values('날짜').copy()
    df['수익률'] = np.log(df['종가'] / df['종가'].shift(1))
    return df

# 벤치마크 수익률 추가
kosdaq_ent = add_return(kosdaq_ent)
kosdaq = add_return(kosdaq)

# 모든 사건 정보 병합
all_incidents = []

for company in companies:
    # 사건 데이터 불러오기
    inc = pd.read_csv(incident_files[company], parse_dates=['사건 날짜'])
    inc['소속사'] = company
    all_incidents.append(inc)

incident_df = pd.concat(all_incidents, ignore_index=True)

# 결과 저장용 리스트
results = []

for company in companies:
    # 주가 데이터 불러오기 & 수익률 계산
    stock = pd.read_csv(stock_files[company], parse_dates=['날짜'])
    stock = add_return(stock)
    
    # 해당 소속사 사건 데이터
    incidents = incident_df[incident_df['소속사'] == company]
    
    for _, row in incidents.iterrows():
        event_date = row['사건 날짜']
        
        # 윈도우 설정 (±7일)
        mask = (stock['날짜'] >= event_date - pd.Timedelta(days=7)) & (stock['날짜'] <= event_date + pd.Timedelta(days=7))
        event_window = stock.loc[mask].copy()

        # 벤치마크도 같은 기간 추출
        bm_ent = kosdaq_ent[kosdaq_ent['날짜'].isin(event_window['날짜'])]
        bm_kosdaq = kosdaq[kosdaq['날짜'].isin(event_window['날짜'])]

        # 병합
        event_window = event_window.merge(bm_ent[['날짜', '수익률']], on='날짜', how='left', suffixes=('', '_KOSDAQ_ENT'))
        event_window = event_window.merge(bm_kosdaq[['날짜', '수익률']], on='날짜', how='left', suffixes=('', '_KOSDAQ'))

        # 초과수익률(AR) 계산
        event_window['AR_KOSDAQ_ENT'] = event_window['수익률'] - event_window['수익률_KOSDAQ_ENT']
        event_window['AR_KOSDAQ'] = event_window['수익률'] - event_window['수익률_KOSDAQ']

        # CAR 계산
        CAR_ent = event_window['AR_KOSDAQ_ENT'].sum(skipna=True)
        CAR_kosdaq = event_window['AR_KOSDAQ'].sum(skipna=True)

        # 결과 저장
        results.append({
            '소속사': company,
            '그룹': row['그룹'],
            '연예인': row['연예인 이름'],
            '사건일': event_date,
            'CAR_KOSDAQ_ENT': CAR_ent,
            'CAR_KOSDAQ': CAR_kosdaq
        })

# 최종 데이터프레임
results_df = pd.DataFrame(results)

# 결과 확인
print(results_df)

# t-test: CAR 평균이 0인가?
for col in ['CAR_KOSDAQ_ENT', 'CAR_KOSDAQ']:
    car_list = results_df[col].dropna()
    t_stat, p_value = stats.ttest_1samp(car_list, 0)
    print(f'\n{col} 검정결과: t={t_stat:.3f}, p={p_value:.3f}, n={len(car_list)}')


      소속사     그룹  연예인        사건일  CAR_KOSDAQ_ENT  CAR_KOSDAQ
0    HYBE  방탄소년단   슈가 2020-05-17        0.000000    0.000000
1    HYBE  방탄소년단   슈가 2020-05-29        0.000000    0.000000
2    HYBE  방탄소년단   RM 2016-07-06        0.000000    0.000000
3    HYBE  방탄소년단   지민 2022-04-25       -0.043926    0.000000
4    HYBE  방탄소년단   정국 2019-09-16        0.000000    0.000000
..    ...    ...  ...        ...             ...         ...
261    YG     위너  송민호 2023-03-24        0.072161    0.000000
262    YG     위너  이승훈 2019-06-12       -0.126170   -0.113548
263    YG     위너  이승훈 2016-08-22       -0.027001   -0.012066
264    YG    트레저  윤재혁 2023-04-02        0.084693    0.000000
265    YG    트레저  하루토 2019-04-06       -0.023170   -0.051496

[266 rows x 6 columns]

CAR_KOSDAQ_ENT 검정결과: t=1.371, p=0.171, n=266

CAR_KOSDAQ 검정결과: t=-1.136, p=0.257, n=266


In [7]:
incident_df.to_csv('../데이터/incidents_merged.csv', index=False, encoding='utf-8-sig')

In [8]:
kosdaq_ent.head()

,날짜,종가,시가,고가,저가,거래량,변동 %,수익률
0,2014-03-10,675.86,656.91,675.86,656.31,17.64K,4.20%,NaN
1,2014-03-11,672.68,679.36,692.03,670.19,30.91K,-0.47%,-0.004716
2,2014-03-12,673.39,677.49,680.32,668.75,33.55K,0.11%,0.001055
3,2014-03-13,679.69,678.71,679.74,661.38,35.02K,0.94%,0.009312
4,2014-03-14,659.55,669.19,677.63,659.55,24.24K,-2.96%,-0.030079


In [9]:
kosdaq.head()

,날짜,종가,시가,고가,저가,거래량,변동 %,수익률
0,2001-01-02,557.0,502.50,559.60,502.5,201.21K,5.93%,NaN
1,2001-01-03,569.0,545.80,579.90,539.3,284.99K,2.15%,0.021315
2,2001-01-04,615.1,617.70,621.50,609.0,233.13K,8.10%,0.077904
3,2001-01-05,625.2,614.50,630.70,602.5,367.23K,1.64%,0.016287
4,2001-01-08,650.0,614.90,657.00,614.0,358.62K,3.97%,0.038901
